<a href="https://colab.research.google.com/github/is0280fp/google_colab/blob/main/cost_sensitive_test_LSTM_5class_stratified_group.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %tensorflow_version 2.x
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# # StratifiedGroupKFoldの分割結果をpickle
import pickle

# test
with open("/content/drive/MyDrive/test_data/V_test.pickle", mode="rb") as f:
   test_V = pickle.load(f)
f.close()
with open("/content/drive/MyDrive/test_data/AP_test.pickle", mode="rb") as f:
   test_AP = pickle.load(f)
f.close()
with open("/content/drive/MyDrive/test_data/ML_test.pickle", mode="rb") as f:
   test_ML = pickle.load(f)
f.close()

with open("/content/drive/MyDrive/test_data/COP_AP_test.pickle", mode="rb") as f:
   test_COP_AP = pickle.load(f)
f.close()
with open("/content/drive/MyDrive/test_data/COP_ML_test.pickle", mode="rb") as f:
   test_COP_ML = pickle.load(f)
f.close()

with open("/content/drive/MyDrive/test_data/label_test.pickle", mode="rb") as f:
   test_label = pickle.load(f)
f.close()

with open("/content/drive/MyDrive/test_data/test_data_index.pickle", mode="rb") as f:
   test_index = pickle.load(f)
f.close()

# trainig and validation
with open("/content/drive/MyDrive/test_data/V_val_train.pickle", mode="rb") as f:
   V = pickle.load(f)
f.close()
with open("/content/drive/MyDrive/test_data/AP_val_train.pickle", mode="rb") as f:
   AP = pickle.load(f)
f.close()
with open("/content/drive/MyDrive/test_data/ML_val_train.pickle", mode="rb") as f:
   ML = pickle.load(f)
f.close()

with open("/content/drive/MyDrive/test_data/COP_AP_val_train.pickle", mode="rb") as f:
   COP_AP = pickle.load(f)
f.close()
with open("/content/drive/MyDrive/test_data/COP_ML_val_train.pickle", mode="rb") as f:
   COP_ML = pickle.load(f)
f.close()

# label
with open("/content/drive/MyDrive/test_data/train_idx_list.pickle", mode="rb") as f:
   train_index_list = pickle.load(f)
f.close()

with open("/content/drive/MyDrive/test_data/val_idx_list.pickle", mode="rb") as f:
   val_index_list = pickle.load(f)
f.close()

with open("/content/drive/MyDrive/test_data/label_val_train.pickle", mode="rb") as f:
   label = pickle.load(f)
f.close()

In [3]:
import numpy as np
print(np.where(label==0)[0].shape)
print(np.where(label==1)[0].shape)
print(np.where(label==2)[0].shape)
print(np.where(label==3)[0].shape)
print(np.where(label==4)[0].shape)

(13966,)
(11969,)
(18135,)
(19398,)
(14853,)


In [4]:
# min_max normalizationする場合
from sklearn import preprocessing
mmscaler_LSTM = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=True) # インスタンスの作成、活性化関数でtanhを使うのでfeature_range=(-1,1) 
mmscaler_CNN = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True) # インスタンスの作成、活性化関数でtanhを使うのでfeature_range=(-1,1)

# LSTM
mmscaler_LSTM.fit(V)
V = mmscaler_LSTM.transform(V)
V = V.reshape(V.shape[0], V.shape[1], 1)

mmscaler_LSTM.fit(AP)
AP = mmscaler_LSTM.transform(AP)
AP = AP.reshape(AP.shape[0], AP.shape[1], 1)

mmscaler_LSTM.fit(ML)
ML = mmscaler_LSTM.transform(ML)
ML = ML.reshape(ML.shape[0], ML.shape[1], 1)

mmscaler_LSTM.fit(COP_AP)
COP_AP = mmscaler_LSTM.transform(COP_AP)
COP_AP = COP_AP.reshape(COP_AP.shape[0], COP_AP.shape[1], 1)

mmscaler_LSTM.fit(COP_ML)
COP_ML = mmscaler_LSTM.transform(COP_ML)
COP_ML = COP_ML.reshape(COP_ML.shape[0], COP_ML.shape[1], 1)

In [5]:
!pip install keras-layer-normalization
# !pip install keras-self-attention

  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.14.0-cp37-none-any.whl size=5269 sha256=060db552ce8b8bb8f4c448ade373af6c7b605dd03f51372b61d887d9dfb66f44
  Stored in directory: /root/.cache/pip/wheels/54/80/22/a638a7d406fd155e507aa33d703e3fa2612b9eb7bb4f4fe667
Successfully built keras-layer-normalization


In [6]:
from keras import optimizers, regularizers
from keras.initializers import he_normal
from keras.layers import ELU, Conv2D, MaxPooling2D, LSTM, InputLayer, Dense, Dropout, Activation, Flatten, concatenate, Conv1D, MaxPooling1D, Input, Reshape, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers.advanced_activations import PReLU
from keras.models import Model, Sequential
from keras.callbacks import TensorBoard, LearningRateScheduler, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from keras.layers.normalization import BatchNormalization
from keras_layer_normalization import LayerNormalization
# from keras_self_attention import SeqSelfAttention
import numpy as np
import keras
from sklearn.utils import class_weight
import tensorflow as tf


import seaborn as sns
import matplotlib.pyplot as plt
import time
import math
import datetime

NAME = "COP_LSTM_5class_stratified_group_with_both"
i = 0

# 共通
now = datetime.datetime.now()

# early stopping
early_stopping = EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.0,
                        patience=10,
                )

# checkpointの設定
checkpoint = ModelCheckpoint(
                    filepath="drive/MyDrive/checkpoints/{}fold_{}.h5".format(i, NAME),
                    monitor='val_loss',
                    save_best_only=True,
                    period=1,
                    verbose=1
                )

initializer = he_normal()

classes_main = 5

#LSTM-----------------------------------------------------------------------------------------------------------------------
def build_model():
  # LSTM
  # 最初これだけだった
  model = Sequential()
  model.add(InputLayer(input_shape=(101, 5)))

  model.add(Bidirectional(LSTM(512, return_sequences=True)))
  # model.add(LSTM(256, return_sequences=True))
  model.add(LayerNormalization())
  model.add(Activation('tanh'))
  model.add(Dropout(0.5))
  
  model.add(Flatten())

  model.add(Dense(512, kernel_initializer=initializer))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.5))

  model.add(Dense(256, kernel_initializer=initializer))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  # model.add(Dropout(0.5))

  model.add(Dense(128, kernel_initializer=initializer))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  # model.add(Dropout(0.5))

  model.add(Dense(64, kernel_initializer=initializer))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  # model.add(Dropout(0.5))

  model.add(Dense(classes_main, kernel_initializer=initializer))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))

  model.summary()
  return model
#------------------------------------------------------------------------------------------------------------------------------------------------------

# k-fold目のモデルのval_accuracyとval_lossのログ
history = []
# 混合行列のログ
cmx_list = []
# F1などスコアのログ
report_list = []

# EPOCHS = 500

val_label_list = []
train_label_list = []

val_LSTM_data_list = []
train_LSTM_data_list = []

i = 0
for i in range(5):
  train_index = train_index_list[i]
  train_LSTM_data = np.concatenate([V[train_index], AP[train_index], ML[train_index], COP_AP[train_index], COP_ML[train_index]], axis=2)
  val_index = val_index_list[i]
  val_LSTM_data = np.concatenate([V[val_index], AP[val_index], ML[val_index], COP_AP[val_index], COP_ML[val_index]], axis=2)
  
  # label
  train_label = label[train_index] 
  val_label= label[val_index] 

  # シャッフル
  p = np.random.permutation(len(train_label))
  train_LSTM_data = train_LSTM_data[p]
  train_label = train_label[p]

  p = np.random.permutation(len(val_label))
  val_LSTM_data = val_LSTM_data[p]
  val_label = val_label[p]

  # MAX = len(train_LSTM_data)

  if i == 0:
    BATCH_SIZE = 55
    # train_LSTM_data = train_LSTM_data[:-3]
    # train_label = train_label[:-3]

  elif i == 1:
    BATCH_SIZE = 55
    train_LSTM_data = train_LSTM_data[:-1]
    train_label = train_label[:-1]

  elif i == 2:
    BATCH_SIZE = 55
    train_LSTM_data = train_LSTM_data[:-1]
    train_label = train_label[:-1]

  elif i == 3:
    BATCH_SIZE = 55
    train_LSTM_data = train_LSTM_data[:-1]
    train_label = train_label[:-1]

  elif i == 4:
    BATCH_SIZE = 55
    train_LSTM_data = train_LSTM_data[:-1]
    train_label = train_label[:-1]




  train_label = keras.utils.to_categorical(train_label, classes_main)
  val_label = keras.utils.to_categorical(val_label, classes_main)

  train_LSTM_data_list.append(train_LSTM_data)
  val_LSTM_data_list.append(val_LSTM_data)

  val_label_list.append(val_label)
  train_label_list.append(train_label)

  print("train_data", train_LSTM_data.shape)
  print("val_data", val_LSTM_data.shape)
  i = i+1

train_data (68530, 101, 5)
val_data (9581, 101, 5)
train_data (68530, 101, 5)
val_data (9012, 101, 5)
train_data (68530, 101, 5)
val_data (9330, 101, 5)
train_data (68530, 101, 5)
val_data (10693, 101, 5)
train_data (68530, 101, 5)
val_data (10993, 101, 5)


In [107]:
def true_positive(sparse_yTrue, sparse_yPred, class_label):
  y_true = K.zeros(len(sparse_yTrue), 1)
  y_true = tf.where(sparse_yTrue == class_label, 1, y_true)
  y_pred = K.zeros(len(sparse_yPred), 1)
  y_pred= tf.where(sparse_yPred == class_label, 1, y_pred)

  # print("sparse_yPred", sparse_yPred)
  # print("y_pred", y_pred)
  tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) # TP
  # y_pos = K.round(K.clip(y_true, 0, 1))
  # n_pos = K.sum(y_pos)
  # y_neg = 1 - y_pos
  # n_neg = K.sum(y_neg)
  # n = n_pos + n_neg
  return tp

def true_negative(sparse_yTrue, sparse_yPred, class_label):
  y_true = K.zeros(len(sparse_yTrue), 1)
  y_true = tf.where(sparse_yTrue == class_label, 1, y_true)
  y_pred = K.zeros(len(sparse_yPred), 1)
  y_pred= tf.where(sparse_yPred == class_label, 1, y_pred)

  y_pos = K.round(K.clip(y_true, 0, 1))
  # n_pos = K.sum(y_pos)
  y_neg = tf.convert_to_tensor(1 - y_pos)
  # n_neg = K.sum(y_neg)
  # n = n_pos + n_neg
  y_pred_pos = K.round(K.clip(y_pred, 0, 1))
  y_pred_neg = tf.convert_to_tensor(1 - y_pred_pos)
  tn = K.sum(K.round(K.clip(y_neg * y_pred_neg, 0, 1))) # TN
  return tn

def false_positive(sparse_yTrue, sparse_yPred, class_label):
  y_true = K.zeros(len(sparse_yTrue), 1)
  y_true = tf.where(sparse_yTrue == class_label, 1, y_true)
  y_pred = K.zeros(len(sparse_yPred), 1)
  y_pred= tf.where(sparse_yPred == class_label, 1, y_pred)

  y_pos = K.round(K.clip(y_true, 0, 1))
  # n_pos = K.sum(y_pos)
  y_neg = tf.convert_to_tensor(1 - y_pos)
  # n_neg = K.sum(y_neg)
  # n = n_pos + n_neg
  fp = K.sum(K.round(K.clip(y_neg * y_pred, 0, 1))) # FP
  return fp

def false_negative(sparse_yTrue, sparse_yPred, class_label):
  # print(len(sparse_yTrue))
  y_true = K.zeros(len(sparse_yTrue), 1)
  y_true = tf.where(sparse_yTrue == class_label, 1, y_true)
  y_pred = K.zeros(len(sparse_yPred), 1)
  y_pred= tf.where(sparse_yPred == class_label, 1, y_pred)

  y_pos = K.round(K.clip(y_true, 0, 1))
  # n_pos = K.sum(y_pos)
  y_neg = tf.convert_to_tensor(1 - y_pos)
  # n_neg = K.sum(y_neg)
  # n = n_pos + n_neg
  y_pred_pos = K.round(K.clip(y_pred, 0, 1))
  y_pred_neg = tf.convert_to_tensor(1 - y_pred_pos)
  fn = K.sum(K.round(K.clip(y_true * y_pred_neg, 0, 1))) # FN
  return fn

# def true_positive(sparse_yTrue, sparse_yPred, class_label):
#   y_true = K.zeros(len(sparse_yTrue), 1)
#   y_true = tf.where(sparse_yTrue == class_label, y_true, 1)
#   y_pred = K.zeros(len(sparse_yPred), 1)
#   y_pred= tf.where(sparse_yPred == class_label, y_pred, 1)
#   tp = 0
#   for i in tf.range(len(y_pred)):
#     if y_true[i] == y_pred[i] == 1:
#       tp += 1
#   tp = tf.convert_to_tensor(tp)
#   # tp = tf.cast(tp, dtype=float64)

#   tp1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1))) # TP
#   print("tp", tp)
#   print("tp1", tp1)

#   return tp

# def true_negative(sparse_yTrue, sparse_yPred, class_label):
#   y_true = K.zeros(len(sparse_yTrue), 1)
#   y_true = tf.where(sparse_yTrue == class_label, y_true, 1)
#   y_pred = K.zeros(len(sparse_yPred), 1)
#   y_pred= tf.where(sparse_yPred == class_label, y_pred, 1)
#   tn = 0
#   for i in tf.range(len(y_pred)):
#     if y_true[i] == y_pred[i] == 0:
#       tn += 1
#   tn = tf.convert_to_tensor(tn)

#   y_pos = K.round(K.clip(y_true, 0, 1))
#   y_neg = tf.convert_to_tensor(1 - y_pos)
#   y_pred_pos = K.round(K.clip(y_pred, 0, 1))
#   y_pred_neg = tf.convert_to_tensor(1 - y_pred_pos)
#   tn1 = K.sum(K.round(K.clip(y_neg * y_pred_neg, 0, 1))) # TN

#   print("tn", tn)
#   print("tn1", tn1)
#   return tn

# def false_positive(sparse_yTrue, sparse_yPred, class_label):
#   y_true = K.zeros(len(sparse_yTrue), 1)
#   y_true = tf.where(sparse_yTrue == class_label, y_true, 1)
#   y_pred = K.zeros(len(sparse_yPred), 1)
#   y_pred= tf.where(sparse_yPred == class_label, y_pred, 1)
#   fp = 0
#   for i in tf.range(len(y_pred)):
#     if y_pred[i] == 1 and y_true[i] != y_pred[i]:
#       fp += 1
#   fp = tf.convert_to_tensor(fp)

#   y_pos = K.round(K.clip(y_true, 0, 1))
#   y_neg = tf.convert_to_tensor(1 - y_pos)
#   fp1 = K.sum(K.round(K.clip(y_neg * y_pred, 0, 1))) # FP

#   print("fp", fp)
#   print("fp1", fp1)
#   return fp

# def false_negative(sparse_yTrue, sparse_yPred, class_label):
#   # print(len(sparse_yTrue))
#   y_true = K.zeros(len(sparse_yTrue), 1)
#   y_true = tf.where(sparse_yTrue == class_label, y_true, 1)
#   y_pred = K.zeros(len(sparse_yPred), 1)
#   y_pred= tf.where(sparse_yPred == class_label, y_pred, 1)
#   fn = 0
#   for i in tf.range(len(y_pred)):
#     if y_pred[i] == 0 and y_true[i] != y_pred[i]:
#       fn += 1
#   fn = tf.convert_to_tensor(fn)

#   y_pos = K.round(K.clip(y_true, 0, 1))
#   y_neg = tf.convert_to_tensor(1 - y_pos)
#   y_pred_pos = K.round(K.clip(y_pred, 0, 1))
#   y_pred_neg = tf.convert_to_tensor(1 - y_pred_pos)
#   fn1 = K.sum(K.round(K.clip(y_true * y_pred_neg, 0, 1))) # FN
  
#   print("fn", fn)
#   print("fn1", fn1)
#   return fn


def values(sparse_yTrue, sparse_yPred, class_label):
  TP = true_positive(sparse_yPred, sparse_yTrue, class_label)
  TP = K.cast(TP, dtype='float64')
  TN = true_negative(sparse_yPred, sparse_yTrue, class_label)
  TN = K.cast(TN, dtype='float64')
  FP = false_positive(sparse_yPred, sparse_yTrue, class_label)
  FP = K.cast(FP, dtype='float64')
  FN = false_negative(sparse_yPred, sparse_yTrue, class_label)
  FN = K.cast(FN, dtype='float64')
  # print("TP", TP)
  # print("FN", FN)
  # print("TN", TN)
  # print("FP", FP)
  # print("1/(TP+FN)", 1/(TP+FN))             # infになる、TP+FN = 0になるとき
  # print("1/(TN+FP)", 1/(TN+FP))             # infになる、TN+FP = 0になるとき
  # print("tf.convert_to_tensor( TP * 1/(TP+FN) * FP * 1/(TN+FP) )", tf.convert_to_tensor( TP * 1/(TP+FN) * FP * 1/(TN+FP) ))
  geometric_value = K.square( tf.convert_to_tensor( TP * 1/(TP+FN) * FP * 1/(TN+FP) ) )
  # print("geometric_value", geometric_value)     # infによってNanになる
  accuracy = tf.convert_to_tensor( (TP + TN) / (TP + TN + FP + FN) )
  return K.cast(tf.convert_to_tensor(geometric_value), dtype='float64'), K.cast(tf.convert_to_tensor(accuracy), dtype='float64')

In [57]:
# from keras import backend as K

# def customLoss(yTrue, yPred):
#   # print("yTrue.shape", yTrue.shape)
#   # print("yPred.shape", yPred.shape)
#   print("yTrue", yTrue)
#   print("yPred", yPred)
#   sparse_yTrue = K.argmax(yTrue, axis=1)           # (50, 1)
#   sparse_yPred = K.argmax(yPred, axis=1)           # (50, 1)
  
#   # from IPython.core.debugger import Pdb; Pdb().set_trace()
  
# #-------あっているはず-----------------------------------------------------------------------------------------------------------------------
#   HC_num = tf.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==0])), dtype=tf.float64)
#   # print("HC_num", HC_num)
#   H_num = tf.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==1])), dtype=tf.float64)
#   K_num = tf.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==2])), dtype=tf.float64)
#   A_num = tf.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==3])), dtype=tf.float64)
#   C_num = tf.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==4])), dtype=tf.float64)

#   HC_IR = tf.convert_to_tensor(HC_num/len(sparse_yTrue[sparse_yTrue!=0]))
#   HC_IR = tf.cast(HC_IR, dtype=tf.float64)
#   H_IR = tf.convert_to_tensor(H_num/len(sparse_yTrue[sparse_yTrue!=1]))
#   H_IR = tf.cast(H_IR, dtype=tf.float64)
#   K_IR = tf.convert_to_tensor(K_num/len(sparse_yTrue[sparse_yTrue!=2]))
#   K_IR = tf.cast(K_IR, dtype=tf.float64)
#   A_IR = tf.convert_to_tensor(A_num/len(sparse_yTrue[sparse_yTrue!=3]))
#   A_IR = tf.cast(A_IR, dtype=tf.float64)
#   C_IR = tf.convert_to_tensor(C_num/len(sparse_yTrue[sparse_yTrue!=4]))
#   C_IR = tf.cast(C_IR, dtype=tf.float64)
# #-------あっているはず-----------------------------------------------------------------------------------------------------------------------

#   # print("HC_IR", HC_IR)          # Nanなし
#   # print("H_IR", H_IR)
#   # print("K_IR", K_IR)
#   # print("A_IR", A_IR)
#   # print("C_IR", C_IR)

#   # print("sparse_yPred_prob", sparse_yPred_prob)
#   HC_idx = tf.where(sparse_yTrue==0)
#   H_idx = tf.where(sparse_yTrue==1)
#   K_idx = tf.where(sparse_yTrue==2)
#   A_idx = tf.where(sparse_yTrue==3)
#   C_idx = tf.where(sparse_yTrue==4)
#   # print("A_idx = tf.where(sparse_yTrue==3)[0]", A_idx)
  
#   HC_sparse_yPred_prob = tf.gather(yPred, HC_idx)
#   H_sparse_yPred_prob = tf.gather(yPred, H_idx)
#   K_sparse_yPred_prob = tf.gather(yPred, K_idx)
#   A_sparse_yPred_prob = tf.gather(yPred, A_idx)
#   C_sparse_yPred_prob = tf.gather(yPred, C_idx)

#   # print("HC_sparse_yPred_prob", HC_sparse_yPred_prob)          # Nanなし
#   # print("H_sparse_yPred_prob", H_sparse_yPred_prob)
#   # print("K_sparse_yPred_prob", K_sparse_yPred_prob)
#   # print("A_sparse_yPred_prob", A_sparse_yPred_prob)
#   # print("C_sparse_yPred_prob", C_sparse_yPred_prob)

#   HC_sparse_yPred_prob = tf.gather(HC_sparse_yPred_prob, indices=0, axis=-1)
#   H_sparse_yPred_prob = tf.gather(H_sparse_yPred_prob, indices=1, axis=-1)
#   K_sparse_yPred_prob = tf.gather(K_sparse_yPred_prob, indices=2, axis=-1)
#   A_sparse_yPred_prob = tf.gather(A_sparse_yPred_prob, indices=3, axis=-1)
#   C_sparse_yPred_prob = tf.gather(C_sparse_yPred_prob, indices=4, axis=-1)

#   # print("HC_sparse_yPred_prob", HC_sparse_yPred_prob)          # Nanなし
#   # print("H_sparse_yPred_prob", H_sparse_yPred_prob)
#   # print("K_sparse_yPred_prob", K_sparse_yPred_prob)
#   # print("A_sparse_yPred_prob", A_sparse_yPred_prob)
#   # print("C_sparse_yPred_prob", C_sparse_yPred_prob)

#   HC_g_values, HC_acc = values(sparse_yTrue, sparse_yPred, 0)
#   H_g_values, H_acc = values(sparse_yTrue, sparse_yPred, 1)
#   K_g_values, K_acc = values(sparse_yTrue, sparse_yPred, 2)
#   A_g_values, A_acc = values(sparse_yTrue, sparse_yPred, 3)
#   C_g_values, C_acc = values(sparse_yTrue, sparse_yPred, 4)

#   # print("HC_g_values", HC_g_values)           # Nanある
#   # print("H_g_values", H_g_values)
#   # print("K_g_values", K_g_values)
#   # print("A_g_values", A_g_values)
#   # print("C_g_values", C_g_values)

#   # print("HC_acc", HC_acc)                     # Nanある
#   # print("H_acc", H_acc)
#   # print("K_acc", K_acc)
#   # print("A_acc", A_acc)
#   # print("C_acc", C_acc)

#   HC_mis_cost = tf.cast(tf.convert_to_tensor( HC_IR * K.exp(- HC_g_values / 2) * K.exp(- HC_acc / 2)), dtype=tf.float64)
#   H_mis_cost =  tf.cast(tf.convert_to_tensor( H_IR * K.exp(- H_g_values / 2) * K.exp(- H_acc / 2)), dtype=tf.float64)
#   K_mis_cost = tf.cast(tf.convert_to_tensor( K_IR * K.exp(- K_g_values / 2) * K.exp(- K_acc / 2)), dtype=tf.float64)
#   A_mis_cost = tf.cast(tf.convert_to_tensor(A_IR * K.exp(- A_g_values / 2) * K.exp(- A_acc / 2)), dtype=tf.float64)
#   C_mis_cost = tf.cast(tf.convert_to_tensor( C_IR * K.exp(- C_g_values / 2) * K.exp(- C_acc / 2)), dtype=tf.float64)

#   # HC_loss = tf.cast(tf.convert_to_tensor(  HC_mis_cost * 1/HC_num * -1 * tf.cast(K.log(HC_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   # H_loss = tf.cast(tf.convert_to_tensor(  H_mis_cost * 1/H_num * -1 * tf.cast(K.log(H_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   # K_loss = tf.cast(tf.convert_to_tensor( K_mis_cost * 1/K_num * -1 * tf.cast(K.log(K_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   # A_loss = tf.cast(tf.convert_to_tensor( A_mis_cost * 1/A_num * -1 * tf.cast(K.log(A_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   # C_loss = tf.cast(tf.convert_to_tensor( C_mis_cost * 1/C_num * -1 * tf.cast(K.log(C_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
  
#   # HC_loss = tf.cast(tf.convert_to_tensor(  HC_mis_cost *  -1 * tf.cast(K.log(HC_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   # H_loss = tf.cast(tf.convert_to_tensor(  H_mis_cost *  -1 * tf.cast(K.log(H_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   # K_loss = tf.cast(tf.convert_to_tensor( K_mis_cost * -1 * tf.cast(K.log(K_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   # A_loss = tf.cast(tf.convert_to_tensor( A_mis_cost *  -1 * tf.cast(K.log(A_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   # C_loss = tf.cast(tf.convert_to_tensor( C_mis_cost *  -1 * tf.cast(K.log(C_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64
  
#   HC_loss = tf.cast(tf.convert_to_tensor(  -1 * tf.cast(tf.math.log(HC_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   H_loss = tf.cast(tf.convert_to_tensor(  -1 * tf.cast(tf.math.log(H_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   K_loss = tf.cast(tf.convert_to_tensor( -1 * tf.cast(tf.math.log(K_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   A_loss = tf.cast(tf.convert_to_tensor( -1 * tf.cast(tf.math.log(A_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)
#   C_loss = tf.cast(tf.convert_to_tensor( -1 * tf.cast(tf.math.log(C_sparse_yPred_prob), dtype=tf.float64)), dtype=tf.float64)

#   # print("HC_sparse_yPred_prob", HC_sparse_yPred_prob)          #Nanなし
#   # print("H_sparse_yPred_prob", H_sparse_yPred_prob)
#   # print("K_sparse_yPred_prob", K_sparse_yPred_prob)
#   # print("A_sparse_yPred_prob", A_sparse_yPred_prob)
#   # print("C_sparse_yPred_prob", C_sparse_yPred_prob)

#   # print("HC_mis_cost",HC_mis_cost)
#   # print("H_mis_cost",H_mis_cost)
#   # print("K_mis_cost",K_mis_cost)
#   # print("A_mis_cost",A_mis_cost)
#   # print("C_mis_cost",C_mis_cost)

#   # print("HC_num", HC_num)
#   # print("H_num", H_num)
#   # print("K_num", K_num)
#   # print("A_num", A_num)
#   # print("C_num", C_num)

#   # print(HC_loss)
#   # print(H_loss)
#   overall_loss = K.concatenate([HC_loss, H_loss, K_loss, A_loss, C_loss], axis=0)
#   # overall_loss = tf.convert_to_tensor(overall_loss * 10)
#   overall_loss = tf.cast(overall_loss, dtype=tf.float64)
#   print("overall_loss", overall_loss)
#   return overall_loss

In [111]:
from keras import backend as K

def miscost(yTrue, yPred):
  # print("yTrue.shape", yTrue.shape)
  # print("yPred.shape", yPred.shape)
  # print("yTrue", yTrue)
  # print("yPred", yPred)
  sparse_yTrue = K.argmax(yTrue, axis=1)           # (50, 1)
  sparse_yPred = K.argmax(yPred, axis=1)           # (50, 1)
  
  HC_num = K.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==0])), dtype='float64')
  H_num = K.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==1])), dtype='float64')
  K_num = K.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==2])), dtype='float64')
  A_num = K.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==3])), dtype='float64')
  C_num = K.cast(tf.convert_to_tensor(len(sparse_yTrue[sparse_yTrue==4])), dtype='float64')

  HC_IR = tf.convert_to_tensor(HC_num/K.cast(len(sparse_yTrue[sparse_yTrue!=0]), dtype='float64'))
  # HC_IR = tf.cast(HC_IR, dtype=tf.float64)
  H_IR = tf.convert_to_tensor(H_num/K.cast(len(sparse_yTrue[sparse_yTrue!=1]), dtype='float64'))
  # H_IR = tf.cast(H_IR, dtype=tf.float64)
  K_IR = tf.convert_to_tensor(K_num/K.cast(len(sparse_yTrue[sparse_yTrue!=2]), dtype='float64'))
  # K_IR = tf.cast(K_IR, dtype=tf.float64)
  A_IR = tf.convert_to_tensor(A_num/K.cast(len(sparse_yTrue[sparse_yTrue!=3]), dtype='float64'))
  # A_IR = tf.cast(A_IR, dtype=tf.float64)
  C_IR = tf.convert_to_tensor(C_num/K.cast(len(sparse_yTrue[sparse_yTrue!=4]), dtype='float64'))
  # C_IR = tf.cast(C_IR, dtype=tf.float64)

  HC_g_values, HC_acc = values(sparse_yTrue, sparse_yPred, 0)
  H_g_values, H_acc = values(sparse_yTrue, sparse_yPred, 1)
  K_g_values, K_acc = values(sparse_yTrue, sparse_yPred, 2)
  A_g_values, A_acc = values(sparse_yTrue, sparse_yPred, 3)
  C_g_values, C_acc = values(sparse_yTrue, sparse_yPred, 4)

  HC_mis_cost = K.cast(tf.convert_to_tensor( HC_IR * K.exp(- HC_g_values / 2) * K.exp(- HC_acc / 2)), dtype='float64')
  H_mis_cost =  K.cast(tf.convert_to_tensor( H_IR * K.exp(- H_g_values / 2) * K.exp(- H_acc / 2)), dtype='float64')
  K_mis_cost = K.cast(tf.convert_to_tensor( K_IR * K.exp(- K_g_values / 2) * K.exp(- K_acc / 2)), dtype='float64')
  A_mis_cost = K.cast(tf.convert_to_tensor(A_IR * K.exp(- A_g_values / 2) * K.exp(- A_acc / 2)), dtype='float64')
  C_mis_cost = K.cast(tf.convert_to_tensor( C_IR * K.exp(- C_g_values / 2) * K.exp(- C_acc / 2)), dtype='float64')
  return HC_mis_cost, H_mis_cost, K_mis_cost, A_mis_cost, C_mis_cost, sparse_yTrue

In [113]:
from keras import backend as K
def customLoss(output, target, from_logits=False):
  _EPSILON = K.epsilon()
  # print(_EPSILON)
  # scale preds so that the class probas of each sample sum to 1
  output /= tf.math.reduce_sum(output,
                          # reduction_indices=len(output.get_shape()) - 1,
                          axis=-1,
                          keepdims=True)
  output = K.cast(output, dtype='float64')
  # print(output)
  epsilon = tf.convert_to_tensor(_EPSILON)
  epsilon = K.cast(epsilon, dtype='float64')
  output = tf.clip_by_value(output, epsilon, 1. - epsilon)
  # print(target)
  # print(output)
  target = K.cast(target, dtype='float64')
  # print(target)
  loss = - tf.math.reduce_sum(target * K.log(output),
                        #  reduction_indices=len(output.get_shape()) - 1
                        axis=-1
                          )
  # HC_mis_cost, H_mis_cost, K_mis_cost, A_mis_cost, C_mis_cost, sparse_yTrue = miscost(output, target)
  # loss_1 = tf.where(sparse_yTrue == 0, tf.convert_to_tensor(loss*HC_mis_cost), loss)
  # loss_1 = tf.where(sparse_yTrue == 1, tf.convert_to_tensor(loss*H_mis_cost), loss_1)
  # loss_1 = tf.where(sparse_yTrue == 2, tf.convert_to_tensor(loss*K_mis_cost), loss_1)
  # loss_1 = tf.where(sparse_yTrue == 3, tf.convert_to_tensor(loss*A_mis_cost), loss_1)
  # loss_1 = tf.where(sparse_yTrue == 4, tf.convert_to_tensor(loss*C_mis_cost), loss_1)
  # print(loss)
  # print(H_mis_cost)
  # print(sparse_yTrue)
  # print(loss_1)
  # return loss_1
  return loss

In [99]:
tf.config.run_functions_eagerly(True)
# tf.config.run_functions_eagerly(False)

In [ ]:
for i in range(0, 5):
  # print("HC", np.where(np.argmax(train_label_list[i], axis=1)==0)[0].shape)
  # print("H", np.where(np.argmax(train_label_list[i], axis=1)==1)[0].shape)
  # print("K", np.where(np.argmax(train_label_list[i], axis=1)==2)[0].shape)
  # print("A", np.where(np.argmax(train_label_list[i], axis=1)==3)[0].shape)
  # print("C", np.where(np.argmax(train_label_list[i], axis=1)==4)[0].shape)

  model = build_model()
  model.compile(
      loss=customLoss,
      # loss = "categorical_crossentropy",
      optimizer= optimizers.Adam(lr=1e-5, beta_1= 0.9, beta_2= 0.999),
      metrics=['acc'],
      # class_weight=class_weight
            )

  result = model.fit(
      x=train_LSTM_data_list[i], 
      y=train_label_list[i], 
      batch_size=BATCH_SIZE, 
      epochs=400, 
      verbose=1,
      validation_data=(
          val_LSTM_data_list[i], 
          val_label_list[i]
            ),
      callbacks=[early_stopping]
      )
  
  print("results per {}-fold".format(i))
  history.append(model.evaluate(
      x=val_LSTM_data_list[i], 
      y=val_label_list[i], 
      verbose=1))
  # save weights
  # now = datetime.datetime.now()
  # file_name = "drive/MyDrive/saved_models/1DCNN_8class_with_both/{}fold_{}_{}".format(i, NAME, now.strftime('%Y%m%d_%H%M%S'))
  file_name = "drive/MyDrive/saved_models/StratifiedGroupFold/COP_LSTM_5class_with_both/{}fold_{}".format(i, NAME)
  model.save(file_name+'.h5')

    # 混合行列
  predict_prob = model.predict(val_LSTM_data_list[i])
  predict_classes=np.argmax(predict_prob,axis=1)
  true_classes = val_label_list[i]
  true_classes=np.argmax(true_classes, axis=1)
  cmx = confusion_matrix(true_classes, predict_classes)
  cmx_list.append(cmx)
  print(cmx)

  index = ["HC", "a_H", "a_K", "a_A", "a_C"]

  report = classification_report(true_classes, predict_classes, target_names=index)
  report_list.append(report)
  print(report)

  cmx_data = []
  for i in range(len(cmx)):
    for j in range(len(cmx)): 
      cmx_data.append((cmx[j, i] / np.sum(cmx[i])))

  cmx_data = np.array(cmx_data).reshape(classes_main, classes_main)

  ax= plt.subplot()
  sns.heatmap(cmx_data, annot=True, ax = ax, fmt="1.3f", cmap="Blues")
  ax.set_xlabel('Predicted labels')
  ax.set_ylabel('True labels')
  ax.set_title('Confusion Matrix') 
  ax.xaxis.set_ticklabels(index) 
  ax.yaxis.set_ticklabels(index)
  plt.show()

  i = i + 1



plt.plot(result.history['acc'])
plt.plot(result.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid()
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid()
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


with open("/content/drive/MyDrive/saved_models/StratifiedGroupFold/COP_LSTM_5class_with_both/cmx_{}_{}.pickle".format(NAME, now.strftime('%Y%m%d_%H%M%S')), mode="wb") as f:
   pickle.dump(cmx_list, f)
f.close()

with open("/content/drive/MyDrive/saved_models/StratifiedGroupFold/COP_LSTM_5class_with_both/classification_report_{}_{}.pickle".format(NAME, now.strftime('%Y%m%d_%H%M%S')), mode="wb") as f:
   pickle.dump(report_list, f)
f.close()

In [ ]:
# min_max normalizationする場合
from sklearn import preprocessing
mmscaler_LSTM = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=True) # インスタンスの作成、活性化関数でtanhを使うのでfeature_range=(-1,1) 
# mmscaler_CNN = preprocessing.MinMaxScaler(feature_range=(0, 1), copy=True) # インスタンスの作成、活性化関数でtanhを使うのでfeature_range=(-1,1)

# LSTM
mmscaler_LSTM.fit(test_V)
test_V = mmscaler_LSTM.transform(test_V)
test_V = test_V.reshape(test_V.shape[0], test_V.shape[1], 1)

mmscaler_LSTM.fit(test_AP)
test_AP = mmscaler_LSTM.transform(test_AP)
test_AP = test_AP.reshape(test_AP.shape[0], test_AP.shape[1], 1)

mmscaler_LSTM.fit(test_ML)
test_ML = mmscaler_LSTM.transform(test_ML)
test_ML = test_ML.reshape(test_ML.shape[0], test_ML.shape[1], 1)

mmscaler_LSTM.fit(test_COP_AP)
test_COP_AP = mmscaler_LSTM.transform(test_COP_AP)
test_COP_AP = test_COP_AP.reshape(test_COP_AP.shape[0], test_COP_AP.shape[1], 1)

mmscaler_LSTM.fit(test_COP_ML)
test_COP_ML = mmscaler_LSTM.transform(test_COP_ML)
test_COP_ML = test_COP_ML.reshape(test_COP_ML.shape[0], test_COP_ML.shape[1], 1)

test_x = np.concatenate([test_V[test_index], test_AP[test_index], test_ML[test_index], test_COP_AP[test_index], test_COP_ML[test_index]], axis=2)
test_y = test_label[test_index]

In [ ]:
# test 
model = build_model()
model.load_weights('/content/drive/MyDrive/saved_models/StratifiedGroupFold/COP_LSTM_5class_with_both/0fold_COP_LSTM_5class_stratified_group_with_both.h5')
model.compile(loss='categorical_crossentropy', 
          optimizer= optimizers.Adam(lr=1e-5, beta_1= 0.9, beta_2= 0.999),
          metrics=['accuracy']
          )
loss, acc = model.evaluate(
    x=test_x, 
    y=test_y,
    verbose=1)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))